## <center> $$\text{Trabalho II}$$
- Dada a gramática LR(1) da Figura 1. Você deve Implementar:
    - (3,5 pontos) Um algoritmo que calcula os conjuntos FISRT e FOLLOW.
    - (3,5 pontos) Um algoritmo que constrói o Autômato LR(1).
    - (3,0 pontos) Um algoritmo para o reconhecimento sintático. Isto é, dada uma palavra w, o seu analisador deve ser capaz de dizer se w obedece ou não as regras da gramática.

In [1]:
from typing import List

In [2]:
A =[1,2,3, 4] 
B = [4]

In [3]:
list(set(A) -set(B))

[1, 2, 3]

In [4]:
class Object():
    """
    Objeto genérico
    """
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            self.key = value

In [47]:
class Language:
    """
    Classe para representar uma linguagem
    """
    def __init__(self, terminals, no_terminals, void_symbol='#', **rules: dict):
        for word, rules in rules.items():
            setattr(self, word, rules)
        self.terminals = terminals
        self.no_terminals = no_terminals
        self.void = void_symbol 

    def first(self, X) -> list:
        first_X = []
        if X in self.terminals:
            return [X]
        if self.void in getattr(self, X):
            first_X.append(self.void)
        print(f'rules: {getattr(self, X)}')
        for rule in getattr(self, X):
            print(f'({X}) rule: {rule}')
            first_Y = self.first(rule[0]) 
            print(f'first({rule[0]}) : {first_Y}')
            first_X += first_Y
            if self.void in first_Y:
                first_X += list(
                    (set(first_Y) - set(self.void)).union(set(self.first(rule[1])))
                )
            i = 0
            while i < len(rule):
                if self.void not in self.first(rule[i]):
                    return first_X
            print(first_X)
            first_X += [self.void]
        print(f'-----{first_X}--------')

In [48]:
lang = Language(
    terminals=['(',')', 'id', '+', '*'],
    no_terminals=['E','S', 'T', 'K', 'F'],
    E = ['TS'],
    S = ['+T S', '#'],
    T = ['F K'],
    K = ['*F K', '#'],
    F = ['(E)', 'id']
)

In [49]:
lang.first('E')

rules: ['TS']
(E) rule: TS
rules: ['F K']
(T) rule: F K
rules: ['(E)', 'id']
(F) rule: (E)
first(() : ['(']
first(F) : ['(']
rules: ['(E)', 'id']
(F) rule: (E)
first(() : ['(']
first(T) : ['(']
rules: ['F K']
(T) rule: F K
rules: ['(E)', 'id']
(F) rule: (E)
first(() : ['(']
first(F) : ['(']
rules: ['(E)', 'id']
(F) rule: (E)
first(() : ['(']


['(']

In [92]:
lang.void

'ε'